# Dash
Dash apps from Jupyter environments: https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e

### Dataset
https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment

### Some inspirations: 
https://www.python-graph-gallery.com/

In [ ]:
import base64
from io import BytesIO
from collections import Counter
import string

import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from wordcloud import WordCloud
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')

# Load and prepare data for visualization

In [ ]:
# Load Data
df = pd.read_csv('Tweets.csv')
df.sample(5)

In [ ]:
# we will use tokenized tweets for word cloud
tweet_tokenizer = TweetTokenizer()

def tokenize(text):
    # skip stop words and punctuation
    stop = set(stopwords.words('english') + list(string.punctuation))
    return [x.lower() for x in tweet_tokenizer.tokenize(text) if x.lower() not in stop]

# add tokenized column to df
df['tokenized'] = df['text'].apply(lambda x: tokenize(x))


# add date column 
df['date'] = pd.DatetimeIndex(df['tweet_created']).date

In [ ]:
# options for our dropdown menu 
sentiments = ['all'] + list(df['airline_sentiment'].unique())
sentiments

In [ ]:
def fillter_by_sentiment(data, sentiment):
    if sentiment == 'all':
        return data
    return data[data['airline_sentiment'] == sentiment]


def word_counter(data):
    cnt = Counter()
    for tokenized in data:
        cnt += Counter(tokenized)
    return cnt

# Dash

https://dash.plotly.com/

In [ ]:
# if building Dash app in script app.py use dash instead of jupyter_dash
# app = dash.Dash(__name__)

# Dash app build in notebook: 
app = JupyterDash(__name__)

## App layout 
Some basic HTML knowledge would be nice: https://www.w3schools.com/html/html_intro.asp

In [ ]:
app.layout = html.Div([
    html.H1("Sentiment Analysis Dashboard"), 
    html.Label([
        "Sentiment type",
        dcc.Dropdown(
            id='sentiments', clearable=False,
            value='all', options=[
                {'label': x, 'value': x}
                for x in sentiments
            ])
    ]),
    html.Div(children=[
        dcc.Graph(id="graph", style={'display': 'inline-block'}),
        html.Img(id="wordcloud", style={'display': 'inline-block'})
    ])
])

## Callbacks 
What and when should change

In [ ]:
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input('sentiments', 'value')]
)
def update_figure(sentiment):
    filltered_df = fillter_by_sentiment(df, sentiment)
    # https://plotly.com/python/histograms/
    return px.histogram(
        filltered_df, x="date", color="airline_sentiment"
    )


# callback to update word cloud
@app.callback(
    Output('wordcloud', 'src'),
    [Input('sentiments', 'value')]
)
def update_wordcloud(sentiment):
    filltered_df = fillter_by_sentiment(df, sentiment)
    img = BytesIO()
    wc = WordCloud(background_color='white', width=480, height=360)
    wc.generate_from_frequencies(word_counter(filltered_df['tokenized']))
    wc.to_image().save(img, format='PNG')
    return 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())

## Run app

In [ ]:
# Run app and display result inline in the notebook
# app.run_server(mode='inline')

# Run app in new window 
app.run_server(mode='external')